## Python imports

In [ ]:
import requests
import json
import os
import sys
import urllib3
urllib3.disable_warnings()

### _Load Django into Jupyter_

In [ ]:
sys.path.append('/bookstore')
sys.path.append('/bookstore/project')
os.environ['DJANGO_SETTINGS_MODULE'] = 'project.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'
import django
django.setup()

## Challenge #9
### Logging to Splunk
Create a token and post a log message

In [ ]:
endpoint = "https://splunk:8089"
auth = ('admin', 'secret123')
app = "bookstore"
index = f"{app}_index"

def get_index_list():
    url = f"{endpoint}/services/data/indexes?datatype=event"
    data = {'output_mode':'json'}
    response = requests.get(url, data=data, auth=auth, verify=False).json()
    return [index['name'] for index in response['entry']]

if index in get_index_list():
    data = {
        'output_mode':'json'
    }
    response = requests.get(f"{endpoint}/services/data/indexes/{index}", data=data, auth=auth, verify=False).json()
else:
    data = {
        'name': index,
        'datatype': 'event',
        'output_mode':'json'
    }
    response = requests.post(f"{endpoint}/services/data/indexes", data=data, auth=auth, verify=False).json()
index = response['entry'][0]
index.pop('acl', None)
index.pop('author', None)
index.pop('content', None)
index.pop('fields', None)
index.pop('links', None)
print(json.dumps(index, indent=4, sort_keys=True))

In [ ]:
def get_input_http_list():
    url = f"{endpoint}/services/data/inputs/http"
    data = {'output_mode':'json'}
    response = requests.get(url, data=data, auth=auth, verify=False).json()
    print()
    return [index['name'] for index in response['entry']]

if f"http://{app}" in get_input_http_list():
    url = f"{endpoint}/servicesNS/nobody/splunk_httpinput/data/inputs/http/http%3A%252F%252F{app}"
    data = {
        'output_mode':'json'
    }
    response = requests.get(url, data=data, auth=auth, verify=False).json()
else:
    data = {
        'name': f'{app}',
        'index': f'{app}_index',
        'description': f'{app.capitalize()} Token',
        'index': f'{app}_index',
        'source': f'{app}_run_monitor',
        'output_mode':'json'
    }
    response = requests.post(f"{endpoint}/services/data/inputs/http", data=data, auth=auth, verify=False).json()
index = response['entry'][0]
index.pop('acl', None)
index.pop('author', None)
index['description'] = index['content']['description']
index['token'] = splunk_token = index['content']['token']
index.pop('content', None)
index.pop('fields', None)
index.pop('links', None)
print(json.dumps(response['entry'][0], indent=4, sort_keys=True))

## Check Collector

In [ ]:
url = "https://splunk:8088/services/collector/health"
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Splunk {splunk_token}'
}

response = requests.get(url, headers=headers, verify=False).json()
print(json.dumps(response, indent=4, sort_keys=True))

## Upload log entries

In [ ]:
import random

def splunklog(dict):
    payload = {
        'index': f'{app}_index',
        'event': dict
    }
    response = requests.post(url, headers=headers, json=payload, verify=False).json()
#     print(json.dumps(response, indent=4, sort_keys=True))
    return response

for i in range(0,4):
    users = ["Anna", "Bob", "Carlos"]
    dict = { 'message' : "Dummy test python client log", 'user': random.choice(users)}
    splunklog(dict)

In [ ]:
https://splunk_server:8089/services/search/jobs/export?search=search index=_internal earliest=-1d latest=now 
        | stats count by host&output_mode=json

In [ ]:
data = {
    'search': 'index="bookstore_index"',
    'output_mode':'json'
}
response = requests.get(f"{endpoint}/services/search/jobs", data=data, auth=auth, verify=False).json()
url = f"{endpoint}{response['entry'][0]['links']['events']}"
response = requests.get(url, data=data, auth=auth, verify=False).json()
for log in response['results']:
    print(json.dumps(log, indent=4, sort_keys=True))